# Se instalan e importan librerias 

In [ ]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=91cd2017a7ba29bdaaeaff672d9817d6edfd40c461156978057c5

In [ ]:
import ee
import json
import geemap
import io
import requests

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Camino al geojson dentro del drive

In [ ]:
geojson_file = '/content/drive/MyDrive/Trabajo/Instacrops/enfermedades/54.geojson'

# Se definen funciones utiles

In [ ]:
# Se extrae la colección de imagenes de acuerdo al poligono del cliente y los rangos de fechas 
def get_sentinel_collection(client_field, dateRange):
    print(dateRange)
    imageCollection = ee.ImageCollection(
        S2.filterBounds(client_field)
          .filterDate(dateRange[0], dateRange[1])
          .filter(ee.Filter.lt('CLOUD_COVER', 35))
          .sort('system:time_start', False)
    )
    return imageCollection

In [ ]:
# Con los feature collection del cliente se genera el poligono con sus coordenadas
def polygon_coordinates_format(FeatureCollection):
  fixCoordinates = [[]]
  for coordinates in FeatureCollection['geometry']['coordinates'][0]:
    fixCoordinates[0].append( [ float(str( coordinates[0])), float(str(coordinates[1])) ] )

  return ee.Geometry.Polygon( fixCoordinates )

In [ ]:
# Se extraen los feature collection del geojson de un cliente
def get_feature_collections(clientMap):
    featureCollections = [ polygon_coordinates_format(feature) for feature in clientMap ]
    return ee.FeatureCollection(featureCollections)

# Se iniciliza libreria Earth Engine

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=iwp2gXxmoL-142IgyQxcrh-1aIcpfSPHwyr3vIBP1TA&tc=hT3pqI6NWeRAmG-Fof-81LcwS0bBZs3ecU2bTGRE2tI&cc=zhUlPZyLDbNgPgUZ2dJN0vTLRay3kWxE0T3wZtPoG10

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VMGYM0BPYwEpp4SY_8tvy0qigDACXGScBtK_aEtrcdFWXo7QorDRNU

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
# link a los dataset: https://developers.google.com/earth-engine/datasets
S2 = ee.ImageCollection("LANDSAT/LC08/C02/T1")

# Se lee poligono del cliente y se extrae colección de imagenes

In [ ]:
f = open(geojson_file)
field_features = json.load(f)

client_field = get_feature_collections(field_features['features'])
imageCollection = get_sentinel_collection(client_field, ('2021-03-15', '2021-04-01'))

('2021-03-15', '2021-04-01')


In [ ]:
imageCollection

# Se guardan bandas B4, B5, B10 y B11 

In [ ]:
img = imageCollection.first()

region = client_field.geometry().dissolve()

for i in ['B4', 'B5', 'B10', 'B11']:

    # Single-band GeoTIFF files wrapped in a zip file.
    url = img.getDownloadUrl({
        'name': 'single_band',
        'bands': i,
        'region': region,
        'crs': 'EPSG:4326',
        'scale': 1
    })
    response = requests.get(url)
    with open(f'single_band_{i}.zip', 'wb') as fd:
      fd.write(response.content)

# Se guarda la banda de elevación

In [ ]:
S3 = ee.ImageCollection([ee.Image('CGIAR/SRTM90_V4')])

imageCollection = ee.ImageCollection(
        S3.filterBounds(client_field)
    )

img = imageCollection.first()

region = client_field.geometry().dissolve()

# Single-band GeoTIFF files wrapped in a zip file.
url = img.getDownloadUrl({
    'name': 'elevation',
    'bands': 'elevation',
    'region': region,
    'crs': 'EPSG:4326',
    'scale': 1
})
response = requests.get(url)
with open(f'single_band.zip', 'wb') as fd:
  fd.write(response.content)